<b>¡Hola Ricardo!</b>

Mi nombre es Alejandro Abia y tengo el gusto de revisar tu proyecto.

A continuación, encontrarás mis comentarios en celdas pintadas de tres colores (verde, amarillo y rojo), a manera de semáforo. Por favor, <b>no las borres ni muevas de posición</b> mientras dure el proceso de revisión.

<div class="alert alert-block alert-success">
<b>Éxito</b> <a class="tocSkip"></a>
En celdas verdes encontrarás comentarios en relación a tus aciertos y fortalezas.
</div>
<div class="alert alert-block alert-warning">
<b>Atención</b> <a class="tocSkip"></a>
Utilizaré el color amarillo para llamar tu atención, expresar algo importante o compartirte alguna idea de valor.
</div>
<div class="alert alert-block alert-danger">
<b>A resolver</b> <a class="tocSkip"></a>
En rojo emitiré aquellos puntos que podrían impedir que el proyecto se ejecute correctamente. No son errores, sino oportunidades importantes de mejora.
</div>
<div class="alert alert-block alert-info">
<b>Comentario estudiante</b><a class="tocSkip"></a>
Si durante la revisión deseas dejarme algún comentario, por favor utiliza celdas azules como esta.
</div>
Tu proyecto será considerado aprobado cuando las observaciones en rojo hayan sido atendidas.
¡Empecemos!


# Proyecto SQL – Análisis de Plataforma de Lectura

## Objetivo del estudio

El propósito de este análisis es explorar una base de datos de una aplicación digital de lectura para generar una propuesta de valor que sirva de base para un nuevo producto en el mercado editorial. Se busca comprender el comportamiento de los usuarios, identificar editoriales y autores destacados, y encontrar patrones de participación que puedan guiar decisiones estratégicas.

## Descripción de los datos

El proyecto se basa en cinco tablas interrelacionadas:

- `books`: contiene información sobre los libros, como número de páginas, fecha de publicación y editorial.
- `authors`: nombres de autores/as y su identificador.
- `publishers`: editoriales registradas.
- `ratings`: calificaciones de los usuarios a los libros.
- `reviews`: reseñas escritas por los usuarios.

## Tareas realizadas

- Contar el número de libros publicados después del año 2000.
- Calcular la cantidad de reseñas y la calificación promedio por libro.
- Identificar la editorial con más libros de más de 50 páginas.
- Encontrar el autor mejor calificado (entre aquellos con mínimo 50 calificaciones).
- Calcular el promedio de reseñas de texto de usuarios que calificaron más de 50 libros.

---

Este análisis fue realizado utilizando consultas SQL y la librería `SQLAlchemy` para conectarse a la base de datos, además de `pandas` para presentar los resultados.



In [1]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

<div class="alert alert-block alert-success">
<b>Celda [1]</b> <a class="tocSkip"></a><br>
Excelente trabajo configurando la conexión a la base de datos. Has utilizado la librería SQLAlchemy de manera efectiva para establecer una conexión segura mediante SSL. Esto es fundamental para proteger los datos durante la transmisión.
</div>


In [2]:
# Muestro las primeras filas de la tabla books
query = 'SELECT * FROM books LIMIT 5;'
df_books = pd.read_sql(query, con=engine)
df_books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<div class="alert alert-block alert-success">
<b>Celda [2]</b> <a class="tocSkip"></a><br>
Buen comienzo al mostrar las primeras filas de la tabla 'books'. Esto te proporciona una visión inicial de la estructura de los datos, lo cual es esencial para planificar los siguientes pasos en el análisis.
</div>


# ¿Cuántos libros se publicaron después del 1 de enero de 2000?

In [3]:
query = '''
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
'''

books_after_2000 = pd.read_sql(query, con=engine)
books_after_2000


,books_after_2000
0,819


<div class="alert alert-block alert-success">
<b>Celda [3]</b> <a class="tocSkip"></a><br>
Has utilizado funciones estadísticas de manera efectiva para obtener una visión inicial del conjunto de datos. Este enfoque es útil para comprender rápidamente el contexto y planificar análisis más detallados.
</div>


### Número de libros publicados después del 1 de enero de 2000
Se encontraron **819 libros** que fueron publicados después del 1 de enero del año 2000.

# Encuenta el número de reseñas de usuarios y la calificación promedio para cada libro.

In [4]:
query = '''
SELECT 
    book_id,
    COUNT(*) AS num_reviews,
    ROUND(AVG(rating), 2) AS avg_rating
FROM ratings
GROUP BY book_id
ORDER BY num_reviews DESC;
'''

book_reviews = pd.read_sql(query, con=engine)
book_reviews.head()


,book_id,num_reviews,avg_rating
0,948,160,3.66
1,750,88,4.13
2,673,86,3.83
3,75,84,3.68
4,302,82,4.41


<div class="alert alert-block alert-success">
<b>Celda [4]</b> <a class="tocSkip"></a><br>
Has calculado el número de reseñas y la calificación promedio para cada libro. 
</div>


In [5]:
query = '''
SELECT 
    b.title,
    r.book_id,
    COUNT(*) AS num_reviews,
    ROUND(AVG(r.rating), 2) AS avg_rating
FROM ratings AS r
JOIN books AS b ON r.book_id = b.book_id
GROUP BY r.book_id, b.title
ORDER BY num_reviews DESC;
'''

book_reviews_named = pd.read_sql(query, con=engine)
book_reviews_named.head()


,title,book_id,num_reviews,avg_rating
0,Twilight (Twilight #1),948,160,3.66
1,The Hobbit or There and Back Again,750,88,4.13
2,The Catcher in the Rye,673,86,3.83
3,Angels & Demons (Robert Langdon #1),75,84,3.68
4,Harry Potter and the Prisoner of Azkaban (Harr...,302,82,4.41


<div class="alert alert-block alert-success">
<b>Celda [5]</b> <a class="tocSkip"></a><br>
Es excelente que hayas incluido los títulos de los libros en tu análisis de reseñas. Esto no solo facilita la interpretación de los datos, sino que también ayuda a los lectores a conectar los resultados con los libros específicos.
</div>


<div class="alert alert-block alert-warning">
<b>Celda [5]</b> <a class="tocSkip"></a><br>
Has agrupado adecuadamente los datos por libro y título.
</div>


### Número de reseñas y calificación promedio por libro

Estos son los libros con mayor número de reseñas:

- **Twilight (Twilight #1)** — 160 reseñas, calificación promedio **3.66**
- **The Hobbit or There and Back Again** — 88 reseñas, calificación promedio **4.13**
- **The Catcher in the Rye** — 86 reseñas, calificación promedio **3.83**
- **Angels & Demons (Robert Langdon #1)** — 84 reseñas, calificación promedio **3.68**
- **Harry Potter and the Prisoner of Azkaban (Harry Potter #3)** — 82 reseñas, calificación promedio **4.41**

Se utilizó la tabla `ratings` agrupando por `book_id` y realizando un `JOIN` con la tabla `books` para obtener los títulos correspondientes.

# Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas

In [6]:
query = '''
SELECT 
    p.publisher, 
    COUNT(b.book_id) AS num_books
FROM books AS b
JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY num_books DESC
LIMIT 1;
'''

pd.read_sql(query, con=engine)

,publisher,num_books
0,Penguin Books,42


<div class="alert alert-block alert-success">
<b>Celda [6]</b> <a class="tocSkip"></a><br>
Has identificado correctamente al editor con más libros de más de 50 páginas. 
</div>


### Editorial con más publicaciones significativas

La editorial que ha publicado la mayor cantidad de libros con más de 50 páginas es:

- **Penguin Books** con **42 libros**.

Este filtro nos ayuda a eliminar folletos u obras menores, centrándonos en publicaciones sustanciales.


# Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.

In [8]:
query = '''
SELECT 
    a.author,
    ROUND(AVG(sub.avg_rating), 2) AS author_avg_rating
FROM (
    SELECT 
        b.book_id,
        b.author_id,
        COUNT(r.rating_id) AS num_ratings,
        AVG(r.rating) AS avg_rating
    FROM books AS b
    JOIN ratings AS r ON b.book_id = r.book_id
    GROUP BY b.book_id, b.author_id
    HAVING COUNT(r.rating_id) >= 50
) AS sub
JOIN authors AS a ON sub.author_id = a.author_id
GROUP BY a.author
ORDER BY author_avg_rating DESC
LIMIT 1;
'''

pd.read_sql(query, con=engine)


,author,author_avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


<div class="alert alert-block alert-success">
<b>Celda [8]</b> <a class="tocSkip"></a><br>
Has calculado de manera precisa la calificación promedio de los autores. 
</div>


### Autor con la calificación promedio más alta

El autor con la calificación promedio más alta (considerando solo libros con al menos 50 calificaciones) es:

- **J.K. Rowling/Mary GrandPré** con un **promedio de 4.28** estrellas.

Este resultado destaca la calidad y popularidad sostenida de sus obras dentro del catálogo.


# Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [9]:
query = '''
WITH active_users AS (
    SELECT 
        username
    FROM ratings
    GROUP BY username
    HAVING COUNT(book_id) > 50
)

SELECT 
    ROUND(AVG(num_reviews), 2) AS avg_text_reviews
FROM (
    SELECT 
        r.username,
        COUNT(rv.review_id) AS num_reviews
    FROM active_users AS r
    JOIN reviews AS rv ON r.username = rv.username
    GROUP BY r.username
) AS user_reviews;
'''

pd.read_sql(query, con=engine)

,avg_text_reviews
0,24.33


<div class="alert alert-block alert-success">
<b>Celda [9]</b> <a class="tocSkip"></a><br>
Has calculado el promedio de reseñas textuales para usuarios activos.
</div>


### Promedio de reseñas entre usuarios activos

Los usuarios que calificaron más de **50 libros** escribieron en promedio:

- **24.33 reseñas de texto**.

Esto indica un alto nivel de compromiso y participación entre los usuarios más activos de la plataforma.


## Conclusiones del Análisis

Este estudio sobre la base de datos de una plataforma de lectura nos permitió generar ideas clave que podrían alimentar una propuesta de valor para un nuevo producto en el mercado editorial digital. A continuación, se resumen los principales hallazgos:

1. **Publicaciones recientes**: Se identificaron **819 libros publicados después del 1 de enero de 2000**, lo que demuestra una oferta amplia y actualizada que puede atraer a nuevos lectores.

2. **Interacción del usuario**: Libros como *"Twilight"*, *"The Hobbit"*, y *"The Catcher in the Rye"* recibieron una gran cantidad de calificaciones y reseñas, lo cual señala su popularidad y valor de marca.

3. **Editorial destacada**: La editorial **Penguin Books** publicó la mayor cantidad de libros con más de 50 páginas, posicionándose como una editorial clave para alianzas o recomendaciones destacadas.

4. **Autor sobresaliente**: **J.K. Rowling/Mary GrandPré** obtuvo la calificación promedio más alta entre los autores con libros altamente evaluados (al menos 50 calificaciones), reflejando una fuerte conexión con la audiencia.

5. **Usuarios comprometidos**: Los usuarios que calificaron más de 50 libros dejaron en promedio **24.33 reseñas de texto**, lo que sugiere una comunidad activa y dispuesta a compartir opiniones útiles.

Estos hallazgos pueden guiar decisiones sobre qué autores y editoriales promover, cómo segmentar audiencias, y qué tipo de contenido priorizar para aumentar la retención y participación de los usuarios.


<div class="alert alert-block alert-success">
<b>Comentario final</b> <a class="tocSkip"></a><br>
¡Muy buen trabajo, Ricardo! A lo largo del proyecto mostraste fortalezas muy claras:<br><br>
• Configuraste correctamente la conexión a la base de datos utilizando SQLAlchemy.<br>
• Realizaste una exploración inicial de los datos mostrando las primeras filas de las tablas, lo cual es esencial para entender la estructura de los datos.<br>
• Incluiste títulos de libros en el análisis de reseñas, mejorando así la comprensión de los resultados.<br>
• Calculaste correctamente métricas clave como el promedio de calificaciones y el número de reseñas.<br><br>
¡Felicidades!
</div>
